In [1]:
from paddleocr import PaddleOCR

import os

image_path = 'sample-images/image_1.jpeg'
ocr = PaddleOCR(
    use_doc_orientation_classify=False, 
    use_doc_unwarping=False, 
    use_textline_orientation=False)


result = ocr.predict(image_path)
for res in result:
    res.print()
    res.save_to_img("output")
    res.save_to_json("output")

/media/edward/SSD-Data/My Folder/ai-receipt-agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Checking connectivity to the model hosters, this may take a while. To bypass this check, set `DISABLE_MODEL_SOURCE_CHECK` to `True`.
/media/edward/SSD-Data/My Folder/ai-receipt-agent/.venv/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/edward/.paddlex/official_models/PP-OCRv5_server_det`.
Cr

In [2]:
import json

with open(os.path.join("output", "image_1_res.json"), "r") as f:
    data = json.load(f)
    print(json.dumps(data, indent=4, ensure_ascii=False))

{
    "input_path": "sample-images/image_1.jpeg",
    "page_index": null,
    "model_settings": {
        "use_doc_preprocessor": false,
        "use_textline_orientation": false
    },
    "dt_polys": [
        [
            [
                211,
                172
            ],
            [
                385,
                174
            ],
            [
                383,
                270
            ],
            [
                209,
                268
            ]
        ],
        [
            [
                225,
                268
            ],
            [
                368,
                271
            ],
            [
                368,
                299
            ],
            [
                225,
                296
            ]
        ],
        [
            [
                156,
                332
            ],
            [
                432,
                336
            ],
            [
                431,
           

In [3]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from openai_function import configure_openai, generate_chat_completion
from dotenv import load_dotenv


load_dotenv()

RECEIPT_SYSTEM_PROMPT="""
Your task is to analyze the receipt text and extract relevant information. Return a JSON object with the following structure: {"items": [{"name": "Item Name", "quantity": 1, "unit_price": 10000, "total_price": 10000}], "subtotal": 20000, "tax": 2000, "ppn": 1000, "service_charge": 500, "other_fees": 0, "total": 23500, "date": "2025-07-08", "merchant": "Merchant Name", "address": "Merchant Address"}. Consider that prices can appear before or after the item names in the text (e.g., both 'Nasi Goreng 25000' and '25000 Nasi Goreng' should be correctly parsed). Normalize quantities (default to 1 if not specified). If any field is missing in the receipt, set its value to null in the JSON. Return only the JSON object, with no extra text or explanation.
"""

def parse_receipt(text):
    client = configure_openai()
    messages = [
        {"role": "system", "content": RECEIPT_SYSTEM_PROMPT},
        {"role": "user", "content": text}
    ]
    
    response = generate_chat_completion(messages)
    
    if response and response.choices:
        content = response.choices[0].message['content']
        try:
            receipt_data = json.loads(content)
            return receipt_data
        except json.JSONDecodeError:
            print("Failed to parse JSON from the response.")
            return None
    else:
        print("No valid response from OpenAI.")
        return None
    
def group_by_lines(detections, y_tolerance=10):
    if not detections:
        return []
    
    # Sort by y-coordinate first
    sorted_detections = sorted(detections, key=lambda x: x[0][0][1])
    
    lines = []
    current_line = [sorted_detections[0]]
    
    for detection in sorted_detections[1:]:
        box, text, confidence = detection
        current_y = box[0][1]  # top-left y coordinate
        last_y = current_line[-1][0][0][1]  # y coordinate of last item in current line
        
        # If y-coordinates are within tolerance, add to current line
        if abs(current_y - last_y) <= y_tolerance:
            current_line.append(detection)
        else:
            # Sort current line by x-coordinate (left to right)
            current_line.sort(key=lambda x: x[0][0][0])
            lines.append(current_line)
            current_line = [detection]
    
    # Don't forget the last line
    if current_line:
        current_line.sort(key=lambda x: x[0][0][0])
        lines.append(current_line)
    
    return lines

In [24]:
result[0]

{'input_path': 'sample-images/image_1.jpeg',
 'page_index': None,
 'doc_preprocessor_res': {'output_img': array([[[148, ..., 191],
          ...,
          [ 33, ...,  25]],
  
         ...,
  
         [[ 47, ...,  62],
          ...,
          [ 57, ..., 130]]], shape=(1280, 720, 3), dtype=uint8)},
 'dt_polys': [array([[211, 172],
         ...,
         [209, 268]], shape=(4, 2), dtype=int16),
  array([[225, 268],
         ...,
         [225, 296]], shape=(4, 2), dtype=int16),
  array([[156, 332],
         ...,
         [155, 359]], shape=(4, 2), dtype=int16),
  array([[241, 358],
         ...,
         [241, 382]], shape=(4, 2), dtype=int16),
  array([[242, 383],
         ...,
         [242, 407]], shape=(4, 2), dtype=int16),
  array([[241, 429],
         ...,
         [242, 454]], shape=(4, 2), dtype=int16),
  array([[154, 477],
         ...,
         [155, 502]], shape=(4, 2), dtype=int16),
  array([[155, 501],
         ...,
         [156, 526]], shape=(4, 2), dtype=int16),
  arra

In [22]:
# Group detections by lines
detections = list(zip(result[0]['rec_polys'], result[0]['rec_texts'], result[0]['rec_scores']))
grouped_lines = group_by_lines(detections, y_tolerance=15)

In [27]:
detections

[(array([[211, 172],
         ...,
         [209, 268]], shape=(4, 2), dtype=int16),
  '感謝',
  0.8780089616775513),
 (array([[225, 268],
         ...,
         [225, 296]], shape=(4, 2), dtype=int16),
  'KANSHA',
  0.9825179576873779),
 (array([[156, 332],
         ...,
         [155, 359]], shape=(4, 2), dtype=int16),
  'Pisa Grande C7 Gading Serpong',
  0.915814995765686),
 (array([[241, 358],
         ...,
         [241, 382]], shape=(4, 2), dtype=int16),
  '085183088285',
  0.9993297457695007),
 (array([[242, 383],
         ...,
         [242, 407]], shape=(4, 2), dtype=int16),
  'Closed Bill',
  0.9429352283477783),
 (array([[241, 429],
         ...,
         [242, 454]], shape=(4, 2), dtype=int16),
  'Table:1/-',
  0.9705137610435486),
 (array([[154, 477],
         ...,
         [155, 502]], shape=(4, 2), dtype=int16),
  '05/07/202512:58',
  0.9810291528701782),
 (array([[155, 501],
         ...,
         [156, 526]], shape=(4, 2), dtype=int16),
  'Order No:PGLtORt',
  0.89838886

In [25]:
# for line_idx, line in enumerate(grouped_lines):
#     # print(f"Line {line_idx + 1}:")
#     for box, text, confidence in line:
#         print(f"{text}")